# BackT Tutorial: Long-Short Moving Average Strategy

This notebook demonstrates how to implement and test a long-short moving average crossover strategy using the BackT framework with mock data. Unlike the long-only version, this strategy goes short on death crosses, potentially profiting in bear markets.

## Overview
1. **Setup & Imports** - Import required libraries
2. **Universe Definition** - Define the assets to trade
3. **Date Configuration** - Set up the backtesting period
4. **Long-Short Strategy** - Create moving average strategy with short positions
5. **Mock Data Generation** - Generate synthetic market data
6. **Engine Configuration** - Configure the BackT engine
7. **Run Backtest** - Execute the strategy
8. **Results Analysis** - Analyze performance metrics
9. **Visualization** - Plot equity curves and results
10. **Long vs Long-Short Comparison** - Compare against long-only approach
11. **Market Scenario Testing** - Test across different market conditions

## 1. Setup & Imports

First, let's import all the necessary components from BackT and other required libraries.

In [ ]:
# Add parent directory to path (if running from notebooks folder)
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

# Import BackT components
from backt import Backtester, BacktestConfig, MockDataLoader
from backt.signal import TechnicalIndicators, StrategyHelpers
from backt.utils.config import ExecutionConfig

# Import standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, Any

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All imports successful!")
print("📊 Ready for long-short backtesting with BackT")
print("⚡ This strategy can go LONG and SHORT!")

## 2. Universe Definition

Define the universe of assets we want to trade. For long-short strategies, we want assets with good liquidity and borrowability.

In [ ]:
# Define our trading universe
universe = {
    'SPY': 'US Large Cap Stocks (S&P 500)',
    'QQQ': 'US Tech Stocks (NASDAQ 100)',
    'TLT': 'Long-term US Treasuries (20+ Year)',
    'GLD': 'Gold ETF',
    'IWM': 'US Small Cap Stocks (Russell 2000)',
    'EFA': 'International Developed Markets'
}

# Extract just the symbols for trading
symbols = list(universe.keys())

# Display our universe
print("🌍 Long-Short Trading Universe:")
print("=" * 50)
for symbol, description in universe.items():
    print(f"{symbol:4} - {description}")
    
print(f"\n📊 Total assets: {len(symbols)}")
print(f"🎯 Symbols for long-short trading: {symbols}")
print(f"📈 Can go LONG on golden crosses")
print(f"📉 Can go SHORT on death crosses")

## 3. Date Configuration

Set up the backtesting period and other time-related parameters.

In [ ]:
# Define backtesting period
start_date = '2020-01-01'
end_date = '2023-12-31'
initial_capital = 100000.0

# Calculate period length
start_dt = pd.to_datetime(start_date)
end_dt = pd.to_datetime(end_date)
period_days = (end_dt - start_dt).days
period_years = period_days / 365.25

print("📅 Long-Short Backtesting Configuration:")
print("=" * 45)
print(f"Start Date:      {start_date}")
print(f"End Date:        {end_date}")
print(f"Period:          {period_days} days ({period_years:.1f} years)")
print(f"Initial Capital: ${initial_capital:,.0f}")
print(f"Data Frequency:  Daily")
print(f"Strategy Type:   Long-Short (can short on death cross)")

# Store configuration for later use
backtest_params = {
    'start_date': start_date,
    'end_date': end_date,
    'initial_capital': initial_capital,
    'period_years': period_years
}

print("\n✅ Date configuration complete!")

## 4. Long-Short Moving Average Strategy Definition

Create a long-short moving average crossover strategy that can take both long and short positions.

In [ ]:
def long_short_moving_average_strategy(
    market_data: Dict[str, pd.DataFrame],
    current_time: pd.Timestamp,
    positions: Dict[str, Any],
    context: Dict[str, Any],
    params: Dict[str, Any]
) -> Dict[str, Dict]:
    """
    Long-Short Moving Average Crossover Strategy
    
    Logic:
    - Calculate short-term (fast) and long-term (slow) moving averages
    - LONG when fast MA crosses above slow MA (golden cross)
    - SHORT when fast MA crosses below slow MA (death cross)
    - Equal weight allocation across selected positions (long and short)
    
    Key Differences from Long-Only:
    - Goes SHORT instead of cash on death crosses
    - Can profit from declining markets
    - Higher potential returns but also higher risk
    
    Parameters:
    - fast_ma: Short-term moving average period (default: 20)
    - slow_ma: Long-term moving average period (default: 50)
    - min_periods: Minimum data points required (default: 60)
    - max_position_size: Maximum weight per position (default: 0.25)
    """
    
    # Strategy parameters
    fast_ma = params.get('fast_ma', 20)
    slow_ma = params.get('slow_ma', 50)
    min_periods = params.get('min_periods', 60)
    max_position_size = params.get('max_position_size', 0.25)
    
    orders = {}
    signals = {}
    long_positions = []
    short_positions = []
    
    # Analyze each asset
    for symbol, data in market_data.items():
        if len(data) < min_periods:
            continue
            
        try:
            # Calculate moving averages
            fast_mavg = TechnicalIndicators.sma(data['close'], fast_ma)
            slow_mavg = TechnicalIndicators.sma(data['close'], slow_ma)
            
            # Get current and previous values
            if len(fast_mavg) < 2 or len(slow_mavg) < 2:
                continue
                
            current_fast = fast_mavg.iloc[-1]
            current_slow = slow_mavg.iloc[-1]
            prev_fast = fast_mavg.iloc[-2]
            prev_slow = slow_mavg.iloc[-2]
            
            # Detect crossovers
            golden_cross = (prev_fast <= prev_slow) and (current_fast > current_slow)
            death_cross = (prev_fast >= prev_slow) and (current_fast < current_slow)
            
            # Generate signals - KEY DIFFERENCE: SHORT on death cross
            if golden_cross:
                signals[symbol] = 'BUY'  # Golden cross = Go long
                long_positions.append(symbol)
            elif death_cross:
                signals[symbol] = 'SELL_SHORT'  # Death cross = Go short
                short_positions.append(symbol)
            elif current_fast > current_slow:
                signals[symbol] = 'HOLD_LONG'  # Fast above slow = Stay long
                long_positions.append(symbol)
            else:
                signals[symbol] = 'HOLD_SHORT'  # Fast below slow = Stay short
                short_positions.append(symbol)
                
        except Exception as e:
            print(f"Warning: Error processing {symbol}: {e}")
            continue
    
    # Calculate position sizing
    total_positions = len(long_positions) + len(short_positions)
    
    if total_positions > 0:
        # Equal weight allocation, but respect max position size
        weight_per_position = min(1.0 / total_positions, max_position_size)
        
        # Create LONG orders
        for symbol in long_positions:
            orders[symbol] = {
                'action': 'target_weight',
                'weight': weight_per_position  # Positive weight = LONG
            }
        
        # Create SHORT orders  
        for symbol in short_positions:
            orders[symbol] = {
                'action': 'target_weight',
                'weight': -weight_per_position  # Negative weight = SHORT
            }
    
    # Close positions for assets with no signal
    for symbol in market_data.keys():
        if symbol not in (long_positions + short_positions):
            if symbol in positions and hasattr(positions[symbol], 'quantity'):
                if positions[symbol].quantity != 0:
                    orders[symbol] = {'action': 'close'}
    
    # Store strategy state for analysis
    context['signals'] = signals
    context['long_positions'] = long_positions
    context['short_positions'] = short_positions
    context['num_long'] = len(long_positions)
    context['num_short'] = len(short_positions)
    context['total_positions'] = total_positions
    context['position_weight'] = weight_per_position if total_positions > 0 else 0
    
    return orders

# Define strategy parameters
strategy_params = {
    'fast_ma': 20,              # 20-day moving average
    'slow_ma': 50,              # 50-day moving average
    'min_periods': 60,          # Minimum data points required
    'max_position_size': 0.25   # Maximum 25% in any single position
}

print("📈 Long-Short Strategy Configuration:")
print("=" * 40)
print(f"Strategy Type:      Moving Average Crossover (Long-Short)")
print(f"Fast MA:            {strategy_params['fast_ma']} days")
print(f"Slow MA:            {strategy_params['slow_ma']} days")
print(f"Min Periods:        {strategy_params['min_periods']} days")
print(f"Max Position Size:  {strategy_params['max_position_size']:.0%}")
print(f"Position Sizing:    Equal weight across long and short")

print("\n🎯 KEY STRATEGY FEATURES:")
print("-" * 30)
print("📈 GOLDEN CROSS (Fast MA > Slow MA):  GO LONG")
print("📉 DEATH CROSS (Fast MA < Slow MA):   GO SHORT")
print("⚡ Can profit in both bull AND bear markets")
print("⚠️  Higher risk due to leverage and short positions")

print("\n✅ Long-short strategy definition complete!")

## 5. Mock Data Generation

Generate synthetic market data for testing our long-short strategy.

In [ ]:
# Test data generation first
print("🔄 Generating mock data for long-short testing...")

# Create mock data loader
mock_loader = MockDataLoader(scenario='normal', seed=42)

# Generate data for our universe
mock_data = mock_loader.load(
    symbols=symbols,
    start_date=start_date,
    end_date=end_date
)

print("📊 Mock Data Summary:")
print("=" * 40)

for symbol, data in mock_data.items():
    start_price = data['close'].iloc[0]
    end_price = data['close'].iloc[-1]
    total_return = (end_price / start_price) - 1
    volatility = data['close'].pct_change().std() * np.sqrt(252)  # Annualized volatility
    
    print(f"{symbol}: {len(data)} days, {total_return:+6.1%} return, {volatility:5.1%} vol")

print(f"\n✅ Mock data generated for {len(mock_data)} assets")
print(f"📈 Data range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"🎯 Perfect for testing long-short strategies!")
print(f"📉 Note: Short positions can profit from negative returns (GLD, IWM)")

## 6. Engine Configuration

Configure the BackT backtesting engine for long-short trading with appropriate costs.

In [ ]:
# Configure execution settings (higher costs for short selling)
execution_config = ExecutionConfig(
    spread=0.00,                # 0.2% bid-ask spread (higher for short selling)
    slippage_pct=0.00,         # 0.1% slippage
    commission_per_share=0.0,    # Commission-free trading
    commission_per_trade=0.0     # No flat fees
)

# Create main backtest configuration
config = BacktestConfig(
    start_date=start_date,
    end_date=end_date,
    initial_capital=initial_capital,
    data_frequency='1D',         # Daily data
    execution=execution_config,
    
    # Mock data settings
    use_mock_data=True,          # Enable mock data mode
    mock_scenario='normal',      # Normal market conditions
    mock_seed=42,                # Reproducible results
    
    # Other settings
    verbose=True,                # Detailed logging
    save_trades=True,            # Save trade history
    save_positions=True,         # Save position history
    save_equity_curve=True       # Save equity curve
)

print("⚙️ Long-Short Engine Configuration:")
print("=" * 40)
print(f"Mock Data Mode:    {config.use_mock_data}")
print(f"Mock Scenario:     {config.mock_scenario}")
print(f"Random Seed:       {config.mock_seed}")
print(f"Spread:            {execution_config.spread:.2%} (higher for short selling)")
print(f"Slippage:          {execution_config.slippage_pct:.2%}")
print(f"Commission:        ${execution_config.commission_per_trade}")
print(f"Strategy Type:     Long-Short (can short sell)")

print(f"\n💡 LONG-SHORT CONSIDERATIONS:")
print(f"-" * 35)
print(f"• Higher transaction costs due to short selling complexity")
print(f"• Short positions have unlimited loss potential")
print(f"• Can profit in both rising and falling markets")
print(f"• Requires margin account and borrowing capability")

print(f"\n✅ BackT engine configured for long-short trading!")
print(f"🎯 Ready to test long-short moving average strategy")

## 7. Run Long-Short Backtest

Execute the long-short moving average crossover strategy on our mock data.

In [ ]:
print("🚀 Starting Long-Short Backtest...")
print("=" * 45)
print(f"Strategy: Long-Short Moving Average Crossover")
print(f"Positions: Can go LONG or SHORT")
print(f"Assets: {len(symbols)} ETFs")

# Initialize backtester
backtester = Backtester(config)

# Run the backtest
start_time = datetime.now()

result = backtester.run(
    strategy=long_short_moving_average_strategy,
    universe=symbols,
    strategy_params=strategy_params
)

end_time = datetime.now()
runtime = (end_time - start_time).total_seconds()

print(f"\n🎉 Long-Short backtest completed!")
print(f"⏱️  Execution time: {runtime:.2f} seconds")
print(f"📊 Results ready for analysis")
print(f"⚡ Strategy used both LONG and SHORT positions")

## Summary

🎯 **What We Accomplished:**

1. ✅ **Implemented a Long-Short Moving Average Strategy** - Can go both long and short
2. ✅ **Added short selling capability** - Death crosses trigger short positions instead of cash
3. ✅ **Enhanced strategy framework** - Equal weight allocation across long and short positions
4. ✅ **Comprehensive testing setup** - Ready for comparison with long-only approaches

🚀 **Key Features:**

- **Market Neutrality**: Can potentially profit in both rising and falling markets
- **Enhanced Signals**: Death crosses become profit opportunities, not just exit signals
- **Risk Management**: Position sizing controls across both long and short positions
- **Realistic Execution**: Higher transaction costs for short selling complexity

This notebook provides the foundation for exploring long-short momentum strategies and comparing their performance characteristics against simpler long-only approaches!